In [1]:
import math
import itertools
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

#### particle vertices, faces, and insphere radius

In [2]:
lattice = gsd.hoomd.open('./DATA/lattice.gsd')[-1]
verts = lattice.particles.type_shapes[0]['vertices']
particle = coxeter.shapes.ConvexPolyhedron(verts)
faces = particle.faces
sigma = 2*particle.insphere_from_center.radius

#### parameters

In [3]:
# integration timestep
dt=0.0005

# coupling
kT_init=1.
kT_second=2.
tau=100*dt

#### simultaion

In [4]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=20)

sim.create_state_from_gsd(filename='./DATA/lattice.gsd')
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                                      kT=kT_init)

#### potential

In [5]:
# neighboring list
nl = hoomd.md.nlist.Cell(buffer=0.4)

# potential
alj = hoomd.md.pair.aniso.ALJ(nl)
alj.r_cut[('A', 'A')] = 2*particle.circumsphere_from_center.radius + 0.15*sigma
alj.params[('A', 'A')] = dict(epsilon=0.1, 
                              sigma_i=sigma, 
                              sigma_j=sigma, 
                              alpha=0)
alj.shape['A'] = dict(vertices=verts, 
                      faces=faces, 
                      rounding_radii=0) # rounding_radii=0.15*(sigma/2)

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/md/pair/aniso.py:589: FutureWarning: 'mode' key is deprectated and will be removed in hoomd 4.0.
  warnings.warn(


#### thermodynamic ensemble

In [6]:
t_duration=int(2e4)

kT = hoomd.variant.Cycle(A=kT_init, 
                         B=kT_second, 
                         t_start=sim.timestep, 
                         t_A=t_duration, 
                         t_AB=t_duration, 
                         t_B=t_duration, 
                         t_BA=t_duration)

nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), 
                           kT=kT, 
                           tau=tau)

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/md/methods/methods.py:102: FutureWarning: NVT is deprecated and wil be removed in hoomd 4.0. In version 4.0, use the ConstantVolume method with the desired thermostat from hoomd.md.methods.thermostats.
  warnings.warn(


#### integrator

In [7]:
integrator = hoomd.md.Integrator(dt=dt, 
                                 methods=[nvt],
                                 forces=[alj],
                                 integrate_rotational_dof=True)

#### writer

In [8]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(alj, quantities=['type_shapes'])
logger.add(thermodynamic_properties)

In [9]:
gsd_writer = hoomd.write.GSD(filename='./DATA/anneal.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             log=logger,
                             mode='wb')

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x11e3e1610> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


In [10]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

#### run

In [11]:
sim.run(0)
alj.type_shapes

[{'type': 'ConvexPolyhedron',
  'rounding_radius': 0.0545562,
  'vertices': [[0.727416, 0, 0.363708],
   [0.363708, 0.629961, 0.363708],
   [-0.363708, 0.629961, 0.363708],
   [-0.727416, 8.90827e-17, 0.363708],
   [-0.363708, -0.629961, 0.363708],
   [0.363708, -0.629961, 0.363708],
   [0.727416, 0, -0.363708],
   [0.363708, 0.629961, -0.363708],
   [-0.363708, 0.629961, -0.363708],
   [-0.727416, 8.90827e-17, -0.363708],
   [-0.363708, -0.629961, -0.363708],
   [0.363708, -0.629961, -0.363708]]}]

In [12]:
sim.run(t_duration*5)